We generated grids on the LA region and we have the weather per day for all the stations in LA.

This script matches each grid on the LA region to the nearest weather station per day.

In [1]:
import pandas as pd

weather_df = pd.read_csv('/mnt/d/airdata/noaa_redownload/la_weather_all_station_per_day.csv')

train_df = pd.read_csv("/mnt/d/airdata/noaa_redownload/la_train_grid.csv")
metadata = pd.read_csv("/mnt/d/airdata/noaa_redownload/la_generated_grid.csv")

In [4]:
generated_grid_df = train_df.merge(metadata, on='wkt')
generated_grid_df['day'] = pd.to_datetime(generated_grid_df['day'])
generated_grid_df

,day,wkt,mean_aod,min_aod,max_aod,Latitudes,Longitudes
0,2018-02-01,POLYGON ((-118.18086152883222 33.7020803924213...,0.151435,0.117,0.193,-118.225777,33.664840
1,2018-02-02,POLYGON ((-118.18086152883222 33.7020803924213...,0.142000,0.103,0.198,-118.225777,33.664840
2,2018-02-03,POLYGON ((-118.18086152883222 33.7020803924213...,0.097443,0.066,0.147,-118.225777,33.664840
3,2018-02-04,POLYGON ((-118.18086152883222 33.7020803924213...,0.098783,0.068,0.138,-118.225777,33.664840
4,2018-02-05,POLYGON ((-118.18086152883222 33.7020803924213...,0.140263,0.103,0.435,-118.225777,33.664840
...,...,...,...,...,...,...,...
469650,2020-12-25,POLYGON ((-117.19271471630032 34.2234442399502...,0.057128,0.024,0.119,-117.237630,34.186204
469651,2020-12-26,POLYGON ((-117.19271471630032 34.2234442399502...,0.047577,0.019,0.105,-117.237630,34.186204
469652,2020-12-29,POLYGON ((-117.19271471630032 34.2234442399502...,0.326000,0.260,0.360,-117.237630,34.186204
469653,2020-12-30,POLYGON ((-117.19271471630032 34.2234442399502...,0.130757,0.110,0.157,-117.237630,34.186204


In [12]:
weather_df['DATE'] = pd.to_datetime(weather_df['DATE'])
weather_df.rename(columns={"longtitude":"latitude", "latitude":"longtitude"}, inplace=True)
weather_df

,STATION,DATE,wind_x,wind_y,ceiling,visibility,temperature,pressure,dew,precipitation_duration,precipitation_depth,longtitude,latitude
0,72032999999,2017-12-07,-19.897360,3.886984,22000.000000,15995.484848,18.318182,NaN,-173.750000,NaN,NaN,34.111542,-117.687589
1,72032999999,2017-12-08,37.933082,26.514359,22000.000000,16067.859375,18.171875,NaN,-144.687500,NaN,NaN,34.111542,-117.687589
2,72032999999,2017-12-09,-15.680682,-28.873602,22000.000000,16048.305556,18.402778,NaN,-119.583333,NaN,NaN,34.111542,-117.687589
3,72032999999,2017-12-10,-8.217215,-8.338019,22000.000000,16093.000000,19.597222,NaN,-143.055556,NaN,NaN,34.111542,-117.687589
4,72032999999,2017-12-11,21.594918,-15.103209,22000.000000,15914.208333,19.416667,NaN,-135.000000,NaN,NaN,34.111542,-117.687589
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57798,A0685400115,2021-12-27,-44.069412,52.424643,17825.458333,28683.863014,-3.958333,NaN,-96.666667,NaN,NaN,34.264000,-116.854000
57799,A0685400115,2021-12-28,-44.658613,26.031856,3933.430556,24789.000000,-2.861111,NaN,-51.666667,NaN,NaN,34.264000,-116.854000
57800,A0685400115,2021-12-29,-37.250340,38.580696,9077.847222,28799.602740,-3.319444,NaN,-57.361111,NaN,NaN,34.264000,-116.854000
57801,A0685400115,2021-12-30,-13.502675,14.835527,1539.426471,26510.043478,-0.161765,NaN,-16.323529,NaN,NaN,34.264000,-116.854000


In [14]:
from math import cos, asin, sqrt

# see: https://en.wikipedia.org/wiki/Haversine_formula
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav))


def lookup(df):
    long = df['Longitudes']
    lati = df['Latitudes']
    day = df['day']
    sliced_day = weather_df[weather_df['DATE'] == day]
    sliced_day['dist'] = sliced_day.apply(lambda x: distance(lati, long, x['latitude'], x['longtitude']), axis=1)
    closest = sliced_day[sliced_day['dist'] == sliced_day['dist'].min()].iloc[0]
    return closest
    # return pd.Series({
    #     "STATION": closest['STATION'].values[0],
    #     "DATE": closest['DATE'].values[0],
    #     "wind_x": closest['wind_x'].values[0],
    #     "wind_y": closest['wind_y'].values[0],
    #     "ceiling": closest['ceiling'].value[0],
    #     "visibility": closest['visibility'].values[0],
    #     "temperature": closest['temperature'].values[0],
    #     "pressure": closest['pressure'].values[0],
    #     "dew": closest['dew'].values[0],
    #     "precipitation_duration": closest['precipitation_duration'].values[0],
    #     "precipitation_depth": closest['precipitation_depth'].values[0],
    #     "latitude": closest['latitude'].values[0],
    #     "longtitude": closest['longtitude'].values[0]
    # })
    

results= generated_grid_df.apply(lookup, axis=1, result_type ='expand')

In [21]:
results

,STATION,DATE,wind_x,wind_y,ceiling,visibility,temperature,pressure,dew,precipitation_duration,precipitation_depth,longtitude,latitude,dist
0,99403599999,2018-02-01,NaN,NaN,NaN,999999.000000,NaN,10150.291667,NaN,NaN,NaN,33.720000,-118.272000,5.902901
1,99403599999,2018-02-02,NaN,NaN,NaN,999999.000000,NaN,10178.625000,NaN,NaN,NaN,33.720000,-118.272000,5.902901
2,99403599999,2018-02-03,NaN,NaN,NaN,999999.000000,NaN,10185.125000,NaN,NaN,NaN,33.720000,-118.272000,5.902901
3,99403599999,2018-02-04,NaN,NaN,NaN,999999.000000,NaN,10172.125000,NaN,NaN,NaN,33.720000,-118.272000,5.902901
4,99403599999,2018-02-05,NaN,NaN,NaN,999999.000000,NaN,10154.958333,NaN,NaN,NaN,33.720000,-118.272000,5.902901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469650,72286699999,2020-12-25,18.794820,15.757901,21012.461538,15474.025641,10.823077,NaN,20.162162,NaN,NaN,34.095353,-117.234875,4.633505
469651,72286699999,2020-12-26,76.872744,-10.518798,20896.333333,15556.555556,11.363889,NaN,-4.305556,NaN,NaN,34.095353,-117.234875,4.633505
469652,72286699999,2020-12-29,-8.882867,-0.025002,15052.780488,14522.951220,7.339024,NaN,21.804878,NaN,NaN,34.095353,-117.234875,4.633505
469653,72286699999,2020-12-30,30.748846,2.748984,22000.000000,15474.025641,13.289744,NaN,-80.473684,NaN,NaN,34.095353,-117.234875,4.633505


In [18]:
generated_grid_df

,day,wkt,mean_aod,min_aod,max_aod,Latitudes,Longitudes
0,2018-02-01,POLYGON ((-118.18086152883222 33.7020803924213...,0.151435,0.117,0.193,-118.225777,33.664840
1,2018-02-02,POLYGON ((-118.18086152883222 33.7020803924213...,0.142000,0.103,0.198,-118.225777,33.664840
2,2018-02-03,POLYGON ((-118.18086152883222 33.7020803924213...,0.097443,0.066,0.147,-118.225777,33.664840
3,2018-02-04,POLYGON ((-118.18086152883222 33.7020803924213...,0.098783,0.068,0.138,-118.225777,33.664840
4,2018-02-05,POLYGON ((-118.18086152883222 33.7020803924213...,0.140263,0.103,0.435,-118.225777,33.664840
...,...,...,...,...,...,...,...
469650,2020-12-25,POLYGON ((-117.19271471630032 34.2234442399502...,0.057128,0.024,0.119,-117.237630,34.186204
469651,2020-12-26,POLYGON ((-117.19271471630032 34.2234442399502...,0.047577,0.019,0.105,-117.237630,34.186204
469652,2020-12-29,POLYGON ((-117.19271471630032 34.2234442399502...,0.326000,0.260,0.360,-117.237630,34.186204
469653,2020-12-30,POLYGON ((-117.19271471630032 34.2234442399502...,0.130757,0.110,0.157,-117.237630,34.186204


In [20]:
pd.concat([generated_grid_df, results], axis=1).to_csv('/mnt/c/Users/Ding/OneDrive - McGill University/Academics/Winter 2022/ECSE 552/proejectdataset/final_data/la_train_grid_with_weather.csv', index=False)

In [15]:
results.to_csv('/mnt/c/Users/Ding/OneDrive - McGill University/Academics/Winter 2022/ECSE 552/proejectdataset/final_data/la_train_grid_with_weather.csv', index=False)